In [2]:
%matplotlib inline

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

from encoder import EncoderRNN
from decoder import LuongAttnDecoderRNN
from movie_line_process import loadLines, loadConversations, extractSentencePairs
from voc import loadPrepareData, trimRareWords, normalizeString
from voc import MIN_COUNT, MAX_LENGTH, PAD_token, SOS_token, EOS_token
from prepare_data import indexesFromSentence, batch2TrainData
from train import trainIters
from model_config import model_name, attn_model, hidden_size
from model_config import encoder_n_layers, decoder_n_layers, dropout, batch_size
from model_config import device, loadFilename, checkpoint_iter
from train_config import clip, learning_rate, decoder_learning_ratio, n_iteration
from train_config import print_every, save_every
from evaluate import GreedySearchDecoder, evaluateInput

In [4]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [5]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

In [7]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)


Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [8]:
# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


In [9]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[7402, 2800, 6883,  900, 6883],
        [4085,  845, 6230, 4350, 3667],
        [7589, 7435, 2154, 2876, 7463],
        [3609,  283, 1887, 7463,  845],
        [ 845, 1183,  637, 4004, 1683],
        [ 907, 7551, 6230, 4513,    2],
        [7435, 4513, 3945,    2,    0],
        [7332, 7419,    2,    0,    0],
        [4513, 1683,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([10, 10,  8,  7,  6])
target_variable: tensor([[7589, 5565, 4958, 2876, 7589],
        [3702,  969,    2, 4513, 6905],
        [4513, 1683,    0,    2, 3803],
        [   2, 7582,    0,    0, 4056],
        [   0, 4513,    0,    0, 4513],
        [   0,    2,    0,    0,    2]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1]], dtype=torch.uint8)
max_target_len: 6


In [10]:
# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [11]:
# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9672
Iteration: 2; Percent complete: 0.1%; Average loss: 8.8414
Iteration: 3; Percent complete: 0.1%; Average loss: 8.6743
Iteration: 4; Percent complete: 0.1%; Average loss: 8.3528
Iteration: 5; Percent complete: 0.1%; Average loss: 7.9295
Iteration: 6; Percent complete: 0.1%; Average loss: 7.3426
Iteration: 7; Percent complete: 0.2%; Average loss: 6.7936
Iteration: 8; Percent complete: 0.2%; Average loss: 6.7994
Iteration: 9; Percent complete: 0.2%; Average loss: 6.8104
Iteration: 10; Percent complete: 0.2%; Average loss: 6.6114
Iteration: 11; Percent complete: 0.3%; Average loss: 6.0325
Iteration: 12; Percent complete: 0.3%; Average loss: 5.8126
Iteration: 13; Percent complete: 0.3%; Average loss: 5.8437
Iteration: 14; Percent complete: 0.4%; Average loss: 5.5351
Iteration: 15; Percent complete: 0.4%; Average loss: 5.7294
Iteration: 16; Percent complete: 0.4%

Iteration: 136; Percent complete: 3.4%; Average loss: 4.0892
Iteration: 137; Percent complete: 3.4%; Average loss: 4.1628
Iteration: 138; Percent complete: 3.5%; Average loss: 4.5382
Iteration: 139; Percent complete: 3.5%; Average loss: 4.4526
Iteration: 140; Percent complete: 3.5%; Average loss: 4.1813
Iteration: 141; Percent complete: 3.5%; Average loss: 4.4443
Iteration: 142; Percent complete: 3.5%; Average loss: 4.2510
Iteration: 143; Percent complete: 3.6%; Average loss: 4.0126
Iteration: 144; Percent complete: 3.6%; Average loss: 4.1882
Iteration: 145; Percent complete: 3.6%; Average loss: 4.3367
Iteration: 146; Percent complete: 3.6%; Average loss: 4.4069
Iteration: 147; Percent complete: 3.7%; Average loss: 4.2325
Iteration: 148; Percent complete: 3.7%; Average loss: 4.2469
Iteration: 149; Percent complete: 3.7%; Average loss: 4.3869
Iteration: 150; Percent complete: 3.8%; Average loss: 4.4267
Iteration: 151; Percent complete: 3.8%; Average loss: 4.1921
Iteration: 152; Percent 

Iteration: 272; Percent complete: 6.8%; Average loss: 3.9734
Iteration: 273; Percent complete: 6.8%; Average loss: 3.9466
Iteration: 274; Percent complete: 6.9%; Average loss: 4.0678
Iteration: 275; Percent complete: 6.9%; Average loss: 3.7542
Iteration: 276; Percent complete: 6.9%; Average loss: 4.3542
Iteration: 277; Percent complete: 6.9%; Average loss: 4.1220
Iteration: 278; Percent complete: 7.0%; Average loss: 4.1248
Iteration: 279; Percent complete: 7.0%; Average loss: 3.7764
Iteration: 280; Percent complete: 7.0%; Average loss: 3.8323
Iteration: 281; Percent complete: 7.0%; Average loss: 3.9930
Iteration: 282; Percent complete: 7.0%; Average loss: 3.9599
Iteration: 283; Percent complete: 7.1%; Average loss: 4.0044
Iteration: 284; Percent complete: 7.1%; Average loss: 4.1102
Iteration: 285; Percent complete: 7.1%; Average loss: 3.6955
Iteration: 286; Percent complete: 7.1%; Average loss: 3.8666
Iteration: 287; Percent complete: 7.2%; Average loss: 3.8124
Iteration: 288; Percent 

Iteration: 408; Percent complete: 10.2%; Average loss: 3.9573
Iteration: 409; Percent complete: 10.2%; Average loss: 3.7308
Iteration: 410; Percent complete: 10.2%; Average loss: 3.7733
Iteration: 411; Percent complete: 10.3%; Average loss: 4.0984
Iteration: 412; Percent complete: 10.3%; Average loss: 3.9018
Iteration: 413; Percent complete: 10.3%; Average loss: 4.1333
Iteration: 414; Percent complete: 10.3%; Average loss: 3.8529
Iteration: 415; Percent complete: 10.4%; Average loss: 3.7287
Iteration: 416; Percent complete: 10.4%; Average loss: 3.9228
Iteration: 417; Percent complete: 10.4%; Average loss: 3.9028
Iteration: 418; Percent complete: 10.4%; Average loss: 3.6317
Iteration: 419; Percent complete: 10.5%; Average loss: 3.7359
Iteration: 420; Percent complete: 10.5%; Average loss: 3.6530
Iteration: 421; Percent complete: 10.5%; Average loss: 3.8707
Iteration: 422; Percent complete: 10.5%; Average loss: 3.5575
Iteration: 423; Percent complete: 10.6%; Average loss: 3.6878
Iteratio

Iteration: 541; Percent complete: 13.5%; Average loss: 3.6803
Iteration: 542; Percent complete: 13.6%; Average loss: 3.7308
Iteration: 543; Percent complete: 13.6%; Average loss: 3.9338
Iteration: 544; Percent complete: 13.6%; Average loss: 3.7486
Iteration: 545; Percent complete: 13.6%; Average loss: 3.5895
Iteration: 546; Percent complete: 13.7%; Average loss: 3.6698
Iteration: 547; Percent complete: 13.7%; Average loss: 3.6503
Iteration: 548; Percent complete: 13.7%; Average loss: 3.5930
Iteration: 549; Percent complete: 13.7%; Average loss: 3.6429
Iteration: 550; Percent complete: 13.8%; Average loss: 3.7918
Iteration: 551; Percent complete: 13.8%; Average loss: 3.5817
Iteration: 552; Percent complete: 13.8%; Average loss: 3.6501
Iteration: 553; Percent complete: 13.8%; Average loss: 3.6722
Iteration: 554; Percent complete: 13.9%; Average loss: 3.5452
Iteration: 555; Percent complete: 13.9%; Average loss: 3.6879
Iteration: 556; Percent complete: 13.9%; Average loss: 3.4671
Iteratio

...

Iteration: 3451; Percent complete: 86.3%; Average loss: 2.7095
Iteration: 3452; Percent complete: 86.3%; Average loss: 2.7979
Iteration: 3453; Percent complete: 86.3%; Average loss: 2.7906
Iteration: 3454; Percent complete: 86.4%; Average loss: 2.8687
Iteration: 3455; Percent complete: 86.4%; Average loss: 2.6362
Iteration: 3456; Percent complete: 86.4%; Average loss: 2.6447
Iteration: 3457; Percent complete: 86.4%; Average loss: 2.8305
Iteration: 3458; Percent complete: 86.5%; Average loss: 2.7644
Iteration: 3459; Percent complete: 86.5%; Average loss: 2.8565
Iteration: 3460; Percent complete: 86.5%; Average loss: 2.6950
Iteration: 3461; Percent complete: 86.5%; Average loss: 2.7161
Iteration: 3462; Percent complete: 86.6%; Average loss: 2.8624
Iteration: 3463; Percent complete: 86.6%; Average loss: 2.8020
Iteration: 3464; Percent complete: 86.6%; Average loss: 2.6686
Iteration: 3465; Percent complete: 86.6%; Average loss: 2.8625
Iteration: 3466; Percent complete: 86.7%; Average loss:

Iteration: 3583; Percent complete: 89.6%; Average loss: 2.7304
Iteration: 3584; Percent complete: 89.6%; Average loss: 2.7884
Iteration: 3585; Percent complete: 89.6%; Average loss: 2.7890
Iteration: 3586; Percent complete: 89.6%; Average loss: 2.8354
Iteration: 3587; Percent complete: 89.7%; Average loss: 2.9144
Iteration: 3588; Percent complete: 89.7%; Average loss: 2.7822
Iteration: 3589; Percent complete: 89.7%; Average loss: 2.7195
Iteration: 3590; Percent complete: 89.8%; Average loss: 2.7865
Iteration: 3591; Percent complete: 89.8%; Average loss: 2.8193
Iteration: 3592; Percent complete: 89.8%; Average loss: 2.5123
Iteration: 3593; Percent complete: 89.8%; Average loss: 2.4664
Iteration: 3594; Percent complete: 89.8%; Average loss: 2.6889
Iteration: 3595; Percent complete: 89.9%; Average loss: 2.8893
Iteration: 3596; Percent complete: 89.9%; Average loss: 2.6172
Iteration: 3597; Percent complete: 89.9%; Average loss: 2.9699
Iteration: 3598; Percent complete: 90.0%; Average loss:

Iteration: 3715; Percent complete: 92.9%; Average loss: 2.7476
Iteration: 3716; Percent complete: 92.9%; Average loss: 2.6863
Iteration: 3717; Percent complete: 92.9%; Average loss: 2.6458
Iteration: 3718; Percent complete: 93.0%; Average loss: 2.4397
Iteration: 3719; Percent complete: 93.0%; Average loss: 2.7951
Iteration: 3720; Percent complete: 93.0%; Average loss: 2.7705
Iteration: 3721; Percent complete: 93.0%; Average loss: 2.7373
Iteration: 3722; Percent complete: 93.0%; Average loss: 2.6062
Iteration: 3723; Percent complete: 93.1%; Average loss: 2.5042
Iteration: 3724; Percent complete: 93.1%; Average loss: 2.6484
Iteration: 3725; Percent complete: 93.1%; Average loss: 2.7070
Iteration: 3726; Percent complete: 93.2%; Average loss: 2.6274
Iteration: 3727; Percent complete: 93.2%; Average loss: 2.7477
Iteration: 3728; Percent complete: 93.2%; Average loss: 2.6696
Iteration: 3729; Percent complete: 93.2%; Average loss: 2.4833
Iteration: 3730; Percent complete: 93.2%; Average loss:

Iteration: 3847; Percent complete: 96.2%; Average loss: 2.7953
Iteration: 3848; Percent complete: 96.2%; Average loss: 2.5719
Iteration: 3849; Percent complete: 96.2%; Average loss: 2.5523
Iteration: 3850; Percent complete: 96.2%; Average loss: 2.4551
Iteration: 3851; Percent complete: 96.3%; Average loss: 2.5660
Iteration: 3852; Percent complete: 96.3%; Average loss: 2.5622
Iteration: 3853; Percent complete: 96.3%; Average loss: 2.6376
Iteration: 3854; Percent complete: 96.4%; Average loss: 2.6488
Iteration: 3855; Percent complete: 96.4%; Average loss: 2.3396
Iteration: 3856; Percent complete: 96.4%; Average loss: 2.7166
Iteration: 3857; Percent complete: 96.4%; Average loss: 2.6506
Iteration: 3858; Percent complete: 96.5%; Average loss: 2.6339
Iteration: 3859; Percent complete: 96.5%; Average loss: 2.6372
Iteration: 3860; Percent complete: 96.5%; Average loss: 2.5091
Iteration: 3861; Percent complete: 96.5%; Average loss: 2.7149
Iteration: 3862; Percent complete: 96.5%; Average loss:

Iteration: 3979; Percent complete: 99.5%; Average loss: 2.7250
Iteration: 3980; Percent complete: 99.5%; Average loss: 2.5929
Iteration: 3981; Percent complete: 99.5%; Average loss: 2.6971
Iteration: 3982; Percent complete: 99.6%; Average loss: 2.5897
Iteration: 3983; Percent complete: 99.6%; Average loss: 2.6827
Iteration: 3984; Percent complete: 99.6%; Average loss: 2.9289
Iteration: 3985; Percent complete: 99.6%; Average loss: 2.6169
Iteration: 3986; Percent complete: 99.7%; Average loss: 2.5391
Iteration: 3987; Percent complete: 99.7%; Average loss: 2.5094
Iteration: 3988; Percent complete: 99.7%; Average loss: 2.9162
Iteration: 3989; Percent complete: 99.7%; Average loss: 2.5455
Iteration: 3990; Percent complete: 99.8%; Average loss: 2.5603
Iteration: 3991; Percent complete: 99.8%; Average loss: 2.5727
Iteration: 3992; Percent complete: 99.8%; Average loss: 2.4838
Iteration: 3993; Percent complete: 99.8%; Average loss: 2.5804
Iteration: 3994; Percent complete: 99.9%; Average loss:

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> hiya
Bot: what ? . . . .
> adsf
Error: Encountered unknown word.
> huh
Bot: what do you mean ? . .
> hiy
Error: Encountered unknown word.
> where is cathyu
Error: Encountered unknown word.
> where is cathy
Bot: where is he ? . . .
> where is bob
Bot: where is he ? . . .
